<a href="https://colab.research.google.com/github/saddam-satria/classification/blob/main/Template_ML_Klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Klasifikasi v1 by Saddam Satria Ardhi 
Digunakan untuk Klasifikasi bukan regresi


Cara pakai
*   Import Package hapus tanda # di bagian paling pertama (!pip)
*   List item
*   Pemilihin Algoritma di import package, tinggal hapus # 
*   Untuk Load data, pilih file train.csv/dataset.csv
*   Untuk penggunaan penghapusan kolom, karena dia inplace, cukup run sekali 
maka kolom sudah terhapus, jika ingin menghapus kolom baru lagi jangan gunakan yang itu, melainkan copas yang baru, lalu isi nama kolom yang ingin dihapus
*   Untuk plotting isi sendiri variabel x dan y nya yang ingin kalian plot, jika tidak digunakan, jangan di run, pasti error, nanti panik lagi
*   tentukan variabel fitur dan target sendiri, fitur adalah x, target adalah y
*   untuk splitting data, atur size nya saja, contoh size = 20, artinya 20% data test
*   bagian Preprocessing, terdapat [] kosong silahkan isi menggunakan string '', pada bagian tersebut sesuaikan dengan kolom kalian, numerik = kolom yang angka, kategorik = kolom yang biasanya huruf
* parameters abaikan saja, jika kalian tidak tahu kegunaannya, jika kalian tahu silahkan isi dengan baik dan benar (dictionary)
* cara mengganti algoritma adalah mengganti nama pada tulisan "model",RandomForestClassifier(), ganti randomforestnya saja "model" tidak perlu di ganti
* jika algoritma kalian ganti maka sesuaikan parameters tadi dengan algoritma kalian, disini ada 2 default parameters dari JCOp untuk Indonesia yaitu gsp dan rsp, gunakan gsp jika kalian menggunakan gridsearchcv, rsp untuk randomizedsearchcv, sesuaikan dengan algoritma kalian rsp.rf_params adalah untuk randomforest paramaters, jika ingin menggunakan yang lain maka ganti rfnya saja atau jika ingin menggunakan grid maka menjadi gsp.rf_params
* hasil evaluasi berupa confusion matrix(false - false +) pelajari cara membacanya
* terdapat baris feature importance, artinya fitur yang mempengaruhi pengurangan score kalian
* Binning isi kolom [''] dengan kolom kalian yang ingin di bining, binds adalah rentang binningnya[0-100] dan labels adalah label dari hasil rentang binningnya, ex: bins[0,12], labels['balita'] artinya 0-12 adalah balita
* Untuk EDA silahkan gunakan section EDA, tidak ada template untuk EDA, karena EDA hasil analisa kalian   




### Import Package 

In [ ]:
#!pip install jcopml <- hapus tanda # sebelum menjalankan

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from google.colab import files
import io

#sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,PowerTransformer,PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

#Algoritma
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier


#Klasifikasi (hapus tanda #)
from sklearn.metrics import accuracy_score,confusion_matrix


#jcopml
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease
from jcopml.tuning import random_search_params as rsp
from jcopml.tuning import grid_search_params as gsp
#Klasifikasi (Hapus tanda #)
from jcopml.plot import plot_confusion_matrix,plot_association_matrix,plot_roc_curve,plot_pr_curve



### Load Data 


In [ ]:
def load_data():
    file = files.upload()
    filename = file.keys()
    for i in (filename):
      global df
      df = pd.read_csv(io.BytesIO(file[i]),index_col='PassengerId')
    return df.head()

load_data()

### Missing Value

In [ ]:
def missing_value(df):
  plot_missing_value(df, figsize=(8,8))
  return df.isna().sum()
missing_value(df)

### Drop Columns

In [ ]:
def remove_columns(kolom):
          df.drop(columns=kolom,inplace=True)
          return df.head()
remove_columns(kolom=[''])

### Plotting

In [ ]:
def plotting(x,y):
    fig, axes = plt.subplots(2, 3, sharex=False, figsize=(20,14))
    fig.suptitle('2 baris 2 kolom')

    sns.scatterplot(ax=axes[0,0],data=df,x=x,y=y)
    sns.histplot(ax=axes[0,1], data=df,x=x,y=y)
    sns.countplot(ax=axes[0,2], data=df,x=x)
    sns.scatterplot(ax=axes[1,0],data=df,x=x,y=y)
    sns.countplot(ax=axes[1,1], data=df,x=x)
    sns.countplot(ax=axes[1,2], data=df,x=x)

plotting('x','y')

### x dan y

In [ ]:
x = df.drop(columns=[''] )
y = df['']

### Splitting Data


In [ ]:
def split_data(x,y,size):
  global xTr,xTs,yTr,yTs
  xTr,xTs,yTr,yTs = train_test_split(x,y, test_size=size)
  return xTr.shape,xTs.shape,yTr.shape,yTs.shape

split_data(x,y,size=30)

### Preprocessing

In [ ]:
xTr.columns

In [ ]:
num_pip = Pipeline([
                    ("Impute", SimpleImputer(strategy="mean")),
                    ("Scale", StandardScaler())

                  ])

cat_pip = Pipeline([
                    ("Impute", SimpleImputer(strategy='most_frequent')),
                    ("OneHotEncode", OneHotEncoder())

                  ])

preprocess = ColumnTransformer([
                                ("numerik",num_pip,['']),
                                ("kategorik",cat_pip,[''])

                              ])

pipeline = Pipeline([
                     ("prep", preprocess),
                     ("model", RandomForestClassifier(n_jobs=-1,verbose=2))

                    ])

### Modeling


In [ ]:
def modeling(xTr,yTr,xTs,yTs,model):
  model.fit(xTr,yTr)
  print(model.best_params_)
  column = [["Validation Score", model.best_score_],["Train Score",model.score(xTr,yTr)],["Test Score",model.score(xTs,yTs)]]
  df = pd.DataFrame(column,columns=['Tipe','Hasil'])
  if (model.score(xTr,yTr)) >= 0.95:
    print("Overfitting")
  elif (model.score(xTr,yTr)) <= 0.65:
    print("Underfitting")
  return df
  

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'model__max_depth':[3,5,10],
    'model__n_estimators':[100,150,200]
    }

In [ ]:
model = RandomizedSearchCV(pipeline,rsp.rf_params,cv=3,n_iter=50,n_jobs=-1,verbose=1)
modeling(xTr,yTr,xTs,yTs,model)

##Table Hasil Iterasi

In [2]:
result = pd.DataFrame(model.cv_results_)

##Plotting Iterasi

In [ ]:
def plot_cv_results(cv_results, param_x, param_z, metric='mean_test_score'):
    """
    cv_results - cv_results_ attribute of a GridSearchCV instance (or similar)
    param_x - name of grid search parameter to plot on x axis
    param_z - name of grid search parameter to plot by line color
    """
    cv_results = pd.DataFrame(cv_results)
    col_x = 'param_' + param_x
    col_z = 'param_' + param_z
    fig, ax = plt.subplots(1, 1, figsize=(11, 8))
    sns.pointplot(x=col_x, y=metric, hue=col_z, data=cv_results, ci=99, n_boot=64, ax=ax)
    ax.set_title("CV Grid Search Results")
    ax.set_xlabel(param_x)
    ax.set_ylabel(metric)
    ax.legend(title=param_z)
    return fig
plot_cv_results(model.cv_results_,'N_Estimator','Gamma',)

### Evaluasi

In [ ]:
def plot_klasifikasi(xTr,yTr,xTs,yTs,model):
    plot_confusion_matrix(xTr,yTr,xTs,yTs,model)
    plot_pr_curve(xTr,yTr,xTs,yTs,model)
    plot_roc_curve(xTr,yTr,xTs,yTs,model)
plot_klasifikasi(xTr,yTr,xTs,yTs,model)

In [ ]:
accuracy_score(xTs,yTs)

### Feature Importance

In [ ]:
df_imp = mean_score_decrease(xTr,yTr,model,plot=True,topk=10)

### Binning

In [ ]:
xTr = pd.cut(xTr[''],bins=[],labels=[])

### EDA

In [ ]:
plot_association_matrix(df,yTr)

In [ ]:
xTr.head()